In [ ]:
!nvidia-smi

Sat Feb  8 05:54:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os


In [ ]:
!pip install ninja

In [ ]:
!pip install .

Processing /content
  Preparing metadata (setup.py) ... done
  Created wheel for cuda_basics: filename=cuda_basics-0.0.5-cp311-cp311-linux_x86_64.whl size=2592950 sha256=7fea829207a643982462230ce4e2d6e709ae4a6321a97837193698768e6b07f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-g1d8nl27/wheels/01/d1/e4/ca90c6fac4331f6da6de5353843d0b67505c2bbc8768ac296e
Successfully built cuda_basics
  Attempting uninstall: cuda_basics
    Found existing installation: cuda_basics 0.0.5
    Uninstalling cuda_basics-0.0.5:
      Successfully uninstalled cuda_basics-0.0.5


In [ ]:
import torch
from gpu_kernels import blur_gpu
import numpy as np
from PIL import Image
import sys
import io

img = Image.open('image.jpg').convert('RGB')
rgb_array = np.array(img)

input_tensor = torch.from_numpy(rgb_array).byte()
input_tensor = input_tensor.reshape(rgb_array.shape[0], rgb_array.shape[1], 3)


output_tensor = torch.zeros_like(input_tensor, dtype=torch.uint8)


#Swap the order of arguments: the input image tensor comes first.
try:
    # Your GPU code here
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()
    blur_gpu(input_tensor, output_tensor)
    output = sys.stdout.getvalue()
    print(output)
    result = Image.fromarray(output_tensor.numpy())
    result.save('blurred_lion.jpeg')
    sys.stdout = old_stdout
    torch.cuda.synchronize()
except Exception as e:
    print(f"Error during GPU operation: {str(e)}")

# result = Image.fromarray(output_tensor.numpy())
# result.save('blur_image.jpg')

In [ ]:
%%writefile blur.cu

#include <opencv2/opencv.hpp>
#include<bits/stdc++.h>
#include <time.h>

__global__ void blur_kernel(unsigned char* Pout, unsigned char* Pin, int width, int height)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;


    if(col < width && row<height)
    {
      // get index given row major format
      int pixVal_r = 0;
      int pixVal_g = 0;
      int pixVal_b = 0;
      int pix_cnt = 0;

      for(int currRow = row - 1;currRow<row+2;currRow++)
      {
        for(int currCol = col - 1;currCol<col+2;currCol++)
        {
            if(currRow < height && currRow > -1 && currCol < width && currCol > -1)
            {
                int index = currRow*width + currCol;

                //intuition : Row 0:   R0  G0  B0   R1  G1  B1   R2  G2  B2   R3  G3  B3

                int rgb_index = index * 3;
                unsigned char r = Pin[rgb_index];
                unsigned char g = Pin[rgb_index + 1];
                unsigned char b = Pin[rgb_index + 2];


                pixVal_r += r;
                pixVal_g += g;
                pixVal_b += b;

                pix_cnt += 1;

            }
        }
      }
        int index = row*width + col;
        int rgb_index = index * 3;
        unsigned char new_r = static_cast<unsigned char> (pixVal_r  / pix_cnt);
        unsigned char new_g = static_cast<unsigned char> (pixVal_g  / pix_cnt);
        unsigned char new_b = static_cast<unsigned char> (pixVal_b  / pix_cnt);


        Pout[rgb_index] =  new_r;
        Pout[rgb_index+1] =  new_g;
        Pout[rgb_index+2] =  new_b;

    }


}

int main() {
    cv::Mat img = cv::imread("image.jpg");

    clock_t start, end;
    start = clock();

    if (img.empty()) {
        std::cerr << "Error: Could not load image!" << std::endl;
        return -1;
    }


    int width = img.cols;
    int height = img.rows;
    int channels = img.channels();


    unsigned char *d_input, *d_output;
    size_t imgSize = width * height * 3 * sizeof(unsigned char);
    size_t blurSize = width * height * 3 * sizeof(unsigned char);

    cudaMalloc((void**)&d_input, imgSize);
    cudaMalloc((void**)&d_output, blurSize);


    cudaMemcpy(d_input,img.data,imgSize,cudaMemcpyHostToDevice);

    cv::Mat blur_img(img.rows, img.cols, CV_8UC3);


    dim3 dimGrid(ceil(width/16.0), ceil(height/16.0), 1);
    dim3 dimBlock(16, 16, 1);

    blur_kernel<<<dimGrid,dimBlock>>>(d_output,d_input,width,height);
    cudaDeviceSynchronize();
    end = clock();
    float time2 = ((float)(end - start)) / CLOCKS_PER_SEC;
    cudaMemcpy(blur_img.data, d_output, blurSize, cudaMemcpyDeviceToHost);



    printf("GPU: %f seconds\n", time2);

    cv::imwrite("blur_image_gpu.jpg", blur_img);

    cudaFree(d_input);
    cudaFree(d_output);

    printf("%d, %d, %d\n",width,height,channels);
    return 0;
}

Overwriting blur.cu


In [ ]:
!nvcc blur.cu -I /usr/include/opencv4/opencv2/  -I/usr/include/opencv4 -L/usr/lib -lopencv_core -lopencv_highgui -lopencv_imgcodecs -o blur -arch=sm_75

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [ ]:
!./blur

GPU: 0.158260 seconds
1879, 2048, 3


In [ ]:
%%writefile blur_cpu.cpp

#include <opencv2/opencv.hpp>
#include<bits/stdc++.h>
#include <time.h>
void blurCPU(unsigned char* Pout, unsigned char* Pin, int width, int height) {
    for (int row = 0; row < height; row++) {
        for (int col = 0; col < width; col++) {
            int pixVal_r = 0, pixVal_g = 0, pixVal_b = 0, pix_cnt = 0;

            for (int blurRow = -1; blurRow <= 1; blurRow++) {
                for (int blurCol = -1; blurCol <= 1; blurCol++) {
                    int curRow = row + blurRow;
                    int curCol = col + blurCol;

                    if (curRow >= 0 && curRow < height && curCol >= 0 && curCol < width) {
                        int rgb_index = (curRow * width + curCol) * 3;

                        unsigned char r = Pin[rgb_index];
                        unsigned char g = Pin[rgb_index + 1];
                        unsigned char b = Pin[rgb_index + 2];

                        pixVal_r += r;
                        pixVal_g += g;
                        pixVal_b += b;
                        pix_cnt++;
                    }
                }
            }


            int rgb_index = (row * width + col) * 3;
            Pout[rgb_index] = static_cast<unsigned char>(pixVal_r / pix_cnt);
            Pout[rgb_index + 1] = static_cast<unsigned char>(pixVal_g / pix_cnt);
            Pout[rgb_index + 2] = static_cast<unsigned char>(pixVal_b / pix_cnt);
        }
    }
}




int main() {

    cv::Mat img = cv::imread("image.jpg");
    if (img.empty()) {
        std::cerr << "Error: Could not load image!" << std::endl;
        return -1;
    }

    clock_t start, end;
    start = clock();

    int width = img.cols;
    int height = img.rows;
    int channels = img.channels();


    cv::Mat blur_img(height, width, CV_8UC3);


    grayscaleCPU(blur_img.data, img.data, width, height);

    end = clock();
    float time2 = ((float)(end - start)) / CLOCKS_PER_SEC;

    printf("CPU: %f seconds\n", time2);

    cv::imwrite("blur_image_cpu.jpg", gray_img);

    printf("%d, %d, %d\n", width, height, channels);
    return 0;
}


Writing vector_addition_cpu.cpp


In [ ]:
!g++ rgb2gray.cpp -I /usr/include/opencv4/opencv2/  -I/usr/include/opencv4 -L/usr/lib -lopencv_core -lopencv_highgui -lopencv_imgcodecs -o cpu_code

cc1plus: fatal error: rgb2gray.cpp: No such file or directory
compilation terminated.


In [ ]:
!./cpu_code

SUCCESS! All values added correctly.
CPU: 1.849633 seconds
